In [1]:
import time
import math
import os
import torch.nn as nn

import torch.onnx
import torch
from torch.autograd import Variable

import model as rnn_model
import data

In [5]:
args_seed = 1234
args_temperature = 1.
args_data = '../data/wikitext-2'
args_model = 'LSTM'
args_emsize = 200
args_nhid = 200
args_nlayers = 2
args_lr = 20
args_clip = 0.25
args_epochs = 40
args_batch_size = 20
args_bptt = 35
args_dropout = 0.2
args_log_interval = 200
args_save = 'model.pt'
args_tied = True

In [24]:
torch.manual_seed(args_seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f'using device: {device}')

using device: cuda


In [25]:
###############################################################################
# Load data
###############################################################################

corpus = data.Corpus(args_data)

In [26]:
# Starting from sequential data, batchify arranges the dataset into columns.
# For instance, with the alphabet as the sequence and batch size 4, we'd get
# ┌ a g m s ┐
# │ b h n t │
# │ c i o u │
# │ d j p v │
# │ e k q w │
# └ f l r x ┘.
# These columns are treated as independent by the model, which means that the
# dependence of e. g. 'g' on 'f' can not be learned, but allows more efficient
# batch processing.

def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

eval_batch_size = 10
train_data = batchify(corpus.train, args_batch_size)
val_data = batchify(corpus.valid, eval_batch_size)
test_data = batchify(corpus.test, eval_batch_size)

In [27]:
###############################################################################
# Build the model
###############################################################################

ntokens = len(corpus.dictionary)
model = rnn_model.RNNModel(args_model, ntokens, args_emsize, args_nhid, args_nlayers, args_dropout, args_tied).to(device)

criterion = nn.CrossEntropyLoss()

In [28]:
###############################################################################
# Training code
###############################################################################

def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

In [29]:
# get_batch subdivides the source data into chunks of length args.bptt.
# If source is equal to the example output of the batchify function, with
# a bptt-limit of 2, we'd get the following two Variables for i = 0:
# ┌ a g m s ┐ ┌ b h n t ┐
# └ b h n t ┘ └ c i o u ┘
# Note that despite the name of the function, the subdivison of data is not
# done along the batch dimension (i.e. dimension 1), since that was handled
# by the batchify function. The chunks are along dimension 0, corresponding
# to the seq_len dimension in the LSTM.

def get_batch(source, i):
    seq_len = min(args_bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target


def evaluate(data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0.
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(eval_batch_size)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, args_bptt):
            data, targets = get_batch(data_source, i)
            output, hidden = model(data, hidden)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
            hidden = repackage_hidden(hidden)
    return total_loss / len(data_source)


def train():
    # Turn on training mode which enables dropout.
    model.train()
    total_loss = 0.
    start_time = time.time()
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(args_batch_size)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, args_bptt)):
        data, targets = get_batch(train_data, i)
        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        hidden = repackage_hidden(hidden)
        model.zero_grad()
        output, hidden = model(data, hidden)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm(model.parameters(), args_clip)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch % args_log_interval == 0 and batch > 0:
            cur_loss = total_loss / args_log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
                    'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // args_bptt, lr,
                elapsed * 1000 / args_log_interval, cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()


In [30]:
# Loop over epochs.
lr = args_lr
best_val_loss = None

# At any point you can hit Ctrl + C to break out of training early.
try:
    for epoch in range(1, args_epochs+1):
        epoch_start_time = time.time()
        train()
        val_loss = evaluate(val_data)
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
                'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                           val_loss, math.exp(val_loss)))
        print('-' * 89)
        # Save the model if the validation loss is the best we've seen so far.
        if not best_val_loss or val_loss < best_val_loss:
            with open(args_save, 'wb') as f:
                torch.save(model, f)
            best_val_loss = val_loss
        else:
            # Anneal the learning rate if no improvement has been seen in the validation dataset.
            lr /= 4.0
except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')

/Users/andrew/miniconda3/envs/breadventure/lib/python3.6/site-packages/ipykernel_launcher.py:52: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


| epoch   1 |   200/ 2983 batches | lr 20.00 | ms/batch 25.36 | loss  7.61 | ppl  2011.34
| epoch   1 |   400/ 2983 batches | lr 20.00 | ms/batch 24.98 | loss  6.76 | ppl   859.31
| epoch   1 |   600/ 2983 batches | lr 20.00 | ms/batch 24.94 | loss  6.35 | ppl   573.49
| epoch   1 |   800/ 2983 batches | lr 20.00 | ms/batch 25.00 | loss  6.19 | ppl   489.42
| epoch   1 |  1000/ 2983 batches | lr 20.00 | ms/batch 25.93 | loss  6.05 | ppl   423.40
| epoch   1 |  1200/ 2983 batches | lr 20.00 | ms/batch 25.36 | loss  5.97 | ppl   391.77
| epoch   1 |  1400/ 2983 batches | lr 20.00 | ms/batch 25.76 | loss  5.86 | ppl   350.52
| epoch   1 |  1600/ 2983 batches | lr 20.00 | ms/batch 25.56 | loss  5.86 | ppl   350.69
| epoch   1 |  1800/ 2983 batches | lr 20.00 | ms/batch 25.70 | loss  5.70 | ppl   299.48
| epoch   1 |  2000/ 2983 batches | lr 20.00 | ms/batch 25.38 | loss  5.67 | ppl   290.72
| epoch   1 |  2200/ 2983 batches | lr 20.00 | ms/batch 25.58 | loss  5.57 | ppl   262.28
| epoch   

| epoch   6 |  1600/ 2983 batches | lr 20.00 | ms/batch 25.39 | loss  4.82 | ppl   124.57
| epoch   6 |  1800/ 2983 batches | lr 20.00 | ms/batch 27.21 | loss  4.71 | ppl   111.21
| epoch   6 |  2000/ 2983 batches | lr 20.00 | ms/batch 28.63 | loss  4.76 | ppl   116.52
| epoch   6 |  2200/ 2983 batches | lr 20.00 | ms/batch 28.37 | loss  4.64 | ppl   103.78
| epoch   6 |  2400/ 2983 batches | lr 20.00 | ms/batch 25.78 | loss  4.69 | ppl   109.37
| epoch   6 |  2600/ 2983 batches | lr 20.00 | ms/batch 26.27 | loss  4.72 | ppl   112.36
| epoch   6 |  2800/ 2983 batches | lr 20.00 | ms/batch 27.10 | loss  4.66 | ppl   105.57
-----------------------------------------------------------------------------------------
| end of epoch   6 | time: 81.94s | valid loss  4.94 | valid ppl   139.71
-----------------------------------------------------------------------------------------
| epoch   7 |   200/ 2983 batches | lr 20.00 | ms/batch 26.11 | loss  4.72 | ppl   111.74
| epoch   7 |   400/ 2983 

| epoch  12 |   200/ 2983 batches | lr 5.00 | ms/batch 26.17 | loss  4.44 | ppl    84.93
| epoch  12 |   400/ 2983 batches | lr 5.00 | ms/batch 25.96 | loss  4.47 | ppl    87.22
| epoch  12 |   600/ 2983 batches | lr 5.00 | ms/batch 26.05 | loss  4.30 | ppl    73.69
| epoch  12 |   800/ 2983 batches | lr 5.00 | ms/batch 27.24 | loss  4.36 | ppl    77.93
| epoch  12 |  1000/ 2983 batches | lr 5.00 | ms/batch 29.61 | loss  4.37 | ppl    79.08
| epoch  12 |  1200/ 2983 batches | lr 5.00 | ms/batch 28.39 | loss  4.38 | ppl    79.48
| epoch  12 |  1400/ 2983 batches | lr 5.00 | ms/batch 28.70 | loss  4.41 | ppl    81.98
| epoch  12 |  1600/ 2983 batches | lr 5.00 | ms/batch 25.62 | loss  4.47 | ppl    87.67
| epoch  12 |  1800/ 2983 batches | lr 5.00 | ms/batch 25.55 | loss  4.36 | ppl    78.64
| epoch  12 |  2000/ 2983 batches | lr 5.00 | ms/batch 25.59 | loss  4.41 | ppl    81.90
| epoch  12 |  2200/ 2983 batches | lr 5.00 | ms/batch 25.64 | loss  4.29 | ppl    72.84
| epoch  12 |  2400/ 

| epoch  17 |  1800/ 2983 batches | lr 5.00 | ms/batch 25.41 | loss  4.29 | ppl    72.67
| epoch  17 |  2000/ 2983 batches | lr 5.00 | ms/batch 25.40 | loss  4.33 | ppl    75.85
| epoch  17 |  2200/ 2983 batches | lr 5.00 | ms/batch 25.38 | loss  4.21 | ppl    67.27
| epoch  17 |  2400/ 2983 batches | lr 5.00 | ms/batch 25.38 | loss  4.25 | ppl    69.93
| epoch  17 |  2600/ 2983 batches | lr 5.00 | ms/batch 25.41 | loss  4.28 | ppl    72.47
| epoch  17 |  2800/ 2983 batches | lr 5.00 | ms/batch 25.41 | loss  4.23 | ppl    68.40
-----------------------------------------------------------------------------------------
| end of epoch  17 | time: 78.55s | valid loss  4.73 | valid ppl   113.63
-----------------------------------------------------------------------------------------
| epoch  18 |   200/ 2983 batches | lr 1.25 | ms/batch 25.71 | loss  4.36 | ppl    77.99
| epoch  18 |   400/ 2983 batches | lr 1.25 | ms/batch 25.37 | loss  4.38 | ppl    80.14
| epoch  18 |   600/ 2983 batches 

| epoch  23 |   200/ 2983 batches | lr 1.25 | ms/batch 25.75 | loss  4.28 | ppl    72.29
| epoch  23 |   400/ 2983 batches | lr 1.25 | ms/batch 26.58 | loss  4.31 | ppl    74.64
| epoch  23 |   600/ 2983 batches | lr 1.25 | ms/batch 27.14 | loss  4.15 | ppl    63.25
| epoch  23 |   800/ 2983 batches | lr 1.25 | ms/batch 27.18 | loss  4.20 | ppl    66.73
| epoch  23 |  1000/ 2983 batches | lr 1.25 | ms/batch 27.17 | loss  4.22 | ppl    68.00
| epoch  23 |  1200/ 2983 batches | lr 1.25 | ms/batch 26.65 | loss  4.23 | ppl    68.51
| epoch  23 |  1400/ 2983 batches | lr 1.25 | ms/batch 26.50 | loss  4.26 | ppl    70.93
| epoch  23 |  1600/ 2983 batches | lr 1.25 | ms/batch 25.97 | loss  4.32 | ppl    75.52
| epoch  23 |  1800/ 2983 batches | lr 1.25 | ms/batch 26.29 | loss  4.22 | ppl    68.13
| epoch  23 |  2000/ 2983 batches | lr 1.25 | ms/batch 25.94 | loss  4.26 | ppl    70.90
| epoch  23 |  2200/ 2983 batches | lr 1.25 | ms/batch 25.42 | loss  4.14 | ppl    62.66
| epoch  23 |  2400/ 

| epoch  28 |  1800/ 2983 batches | lr 0.31 | ms/batch 25.40 | loss  4.20 | ppl    66.93
| epoch  28 |  2000/ 2983 batches | lr 0.31 | ms/batch 25.42 | loss  4.24 | ppl    69.50
| epoch  28 |  2200/ 2983 batches | lr 0.31 | ms/batch 25.76 | loss  4.12 | ppl    61.57
| epoch  28 |  2400/ 2983 batches | lr 0.31 | ms/batch 25.43 | loss  4.16 | ppl    63.81
| epoch  28 |  2600/ 2983 batches | lr 0.31 | ms/batch 26.03 | loss  4.20 | ppl    66.44
| epoch  28 |  2800/ 2983 batches | lr 0.31 | ms/batch 25.67 | loss  4.13 | ppl    61.93
-----------------------------------------------------------------------------------------
| end of epoch  28 | time: 79.23s | valid loss  4.66 | valid ppl   105.98
-----------------------------------------------------------------------------------------
| epoch  29 |   200/ 2983 batches | lr 0.31 | ms/batch 25.87 | loss  4.27 | ppl    71.23
| epoch  29 |   400/ 2983 batches | lr 0.31 | ms/batch 27.26 | loss  4.30 | ppl    73.48
| epoch  29 |   600/ 2983 batches 

| epoch  34 |   200/ 2983 batches | lr 0.08 | ms/batch 25.57 | loss  4.26 | ppl    71.09
| epoch  34 |   400/ 2983 batches | lr 0.08 | ms/batch 25.48 | loss  4.29 | ppl    73.22
| epoch  34 |   600/ 2983 batches | lr 0.08 | ms/batch 25.50 | loss  4.12 | ppl    61.80
| epoch  34 |   800/ 2983 batches | lr 0.08 | ms/batch 25.63 | loss  4.19 | ppl    66.12
| epoch  34 |  1000/ 2983 batches | lr 0.08 | ms/batch 25.44 | loss  4.20 | ppl    66.85
| epoch  34 |  1200/ 2983 batches | lr 0.08 | ms/batch 25.50 | loss  4.21 | ppl    67.17
| epoch  34 |  1400/ 2983 batches | lr 0.08 | ms/batch 25.67 | loss  4.24 | ppl    69.26
| epoch  34 |  1600/ 2983 batches | lr 0.08 | ms/batch 25.59 | loss  4.30 | ppl    73.52
| epoch  34 |  1800/ 2983 batches | lr 0.08 | ms/batch 25.61 | loss  4.20 | ppl    66.75
| epoch  34 |  2000/ 2983 batches | lr 0.08 | ms/batch 25.51 | loss  4.24 | ppl    69.38
| epoch  34 |  2200/ 2983 batches | lr 0.08 | ms/batch 25.64 | loss  4.11 | ppl    61.20
| epoch  34 |  2400/ 

| epoch  39 |  1800/ 2983 batches | lr 0.08 | ms/batch 25.34 | loss  4.20 | ppl    66.51
| epoch  39 |  2000/ 2983 batches | lr 0.08 | ms/batch 25.34 | loss  4.24 | ppl    69.23
| epoch  39 |  2200/ 2983 batches | lr 0.08 | ms/batch 25.34 | loss  4.12 | ppl    61.42
| epoch  39 |  2400/ 2983 batches | lr 0.08 | ms/batch 25.35 | loss  4.15 | ppl    63.39
| epoch  39 |  2600/ 2983 batches | lr 0.08 | ms/batch 25.36 | loss  4.19 | ppl    65.93
| epoch  39 |  2800/ 2983 batches | lr 0.08 | ms/batch 25.36 | loss  4.12 | ppl    61.79
-----------------------------------------------------------------------------------------
| end of epoch  39 | time: 78.37s | valid loss  4.66 | valid ppl   105.41
-----------------------------------------------------------------------------------------
| epoch  40 |   200/ 2983 batches | lr 0.08 | ms/batch 25.56 | loss  4.26 | ppl    70.54
| epoch  40 |   400/ 2983 batches | lr 0.08 | ms/batch 25.39 | loss  4.29 | ppl    73.07
| epoch  40 |   600/ 2983 batches 

In [31]:
# Load the best saved model.
with open(args_save, 'rb') as f:
    model = torch.load(f)
    # after load the rnn params are not a continuous chunk of memory
    # this makes them a continuous chunk, and will speed up forward pass
    model.rnn.flatten_parameters()

# Run on test data.
test_loss = evaluate(test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)


| End of training | test loss  4.61 | test ppl   100.09


# Try to generate with this model now

In [2]:
args_data = '../data/wikitext-2'
args_checkpoint = './model.pt'
args_outf = 'generated.txt'
args_words = 1000
args_seed = 1234
args_temperature = 1.0
args_log_interval = 100

In [3]:
# Set the random seed manually for reproducibility.
torch.manual_seed(args_seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cuda


In [5]:
if args_temperature < 1e-3:
    print("args_temperature has to be greater or equal 1e-3")

with open(args_checkpoint, 'rb') as f:
    model = torch.load(f).to(device)
    

In [6]:
model.eval()

RNNModel(
  (drop): Dropout(p=0.2)
  (encoder): Embedding(33278, 200)
  (rnn): LSTM(200, 200, num_layers=2, dropout=0.2)
  (decoder): Linear(in_features=200, out_features=33278, bias=True)
)

In [7]:
corpus = data.Corpus(args_data)
ntokens = len(corpus.dictionary)
hidden = model.init_hidden(1)
input = torch.randint(ntokens, (1, 1), dtype=torch.long).to(device)

In [8]:
with open(args_outf, 'w') as outf:
    with torch.no_grad():  # no tracking history
        for i in range(args_words):
            output, hidden = model(input, hidden)
            word_weights = output.squeeze().div(args_temperature).exp().cpu()
            word_idx = torch.multinomial(word_weights, 1)[0]
            input.fill_(word_idx)
            word = corpus.dictionary.idx2word[word_idx]

            outf.write(word + ('\n' if i % 20 == 19 else ' '))

            if i % args_log_interval == 0:
                print('| Generated {}/{} words'.format(i, args_words))

| Generated 0/1000 words
| Generated 100/1000 words
| Generated 200/1000 words
| Generated 300/1000 words
| Generated 400/1000 words
| Generated 500/1000 words
| Generated 600/1000 words
| Generated 700/1000 words
| Generated 800/1000 words
| Generated 900/1000 words


In [15]:
model.state_dict()

torch.Size([33278, 200])

In [10]:
## Save State Dictionary
with open('model_state.pt', 'wb') as f:
    torch.save(model.state_dict(), f)
